In [1]:
import nbaPlayerStatModeling as nbaModel

In [ ]:
nba = nbaModel.playerStatModel(
        day_offset = 0, 
        season = '2024-25', 
        perMode = 'PerGame',
        num_simulations = 200000
)

In [3]:
nba.get_teams_playing()
nba.get_teams_data(
    base_url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType={stattype}&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
)
nba.get_players_playing(
    url_base_nba_player_stat = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID={tid}&TwoWay=0&VsConference=&VsDivision=&Weight=',
    minute_cutoff = 0
)
nba.get_props()

4  games today...
138 players returned from 8 teams, out of a total 8
635 prop bets for 66 players...


In [7]:
#nba.df_players = nba.df_players.iloc[[1,10,19,25,30], :]
nba.get_player_home_adv(
    use_default=False
)

1 / 5 player splits..
2 / 5 player splits..
3 / 5 player splits..
4 / 5 player splits..
5 / 5 player splits..


In [64]:
#df_copy = nba.df_players.copy()
nba.df_players = df_copy.copy()

In [ ]:
nba.model_expected_minutes()
nba.calculate_model_inputs()
nba.df_players = nba.df_players.fillna(0)
#nba.num_simulations = 250000
nba.model_expected_stats_poisson()

In [66]:
nba.model_expected_pts()

In [67]:
nba.df_players[['PLAYER_NAME', 'DREB_HOMEadj']]

,PLAYER_NAME,DREB_HOMEadj
0,Coby White,0.902778
1,Josh Hart,1.037853
2,Bennedict Mathurin,1.200000
3,Tyrese Haliburton,1.058537
4,Quentin Grimes,1.051282


In [68]:
nba.df_players[['PLAYER_NAME', 'REB', 'expReb', 'reb_line', 'reb_oOdds', 'REBoOdds','REBoOdds_deci','REBoProb']]

,PLAYER_NAME,REB,expReb,reb_line,reb_oOdds,REBoOdds,REBoOdds_deci,REBoProb
0,Coby White,3.3,3.047735,3.5,120.0,174,2.748,0.363848
1,Josh Hart,9.6,10.512123,9.5,-122.0,-152,1.656,0.603792
2,Bennedict Mathurin,5.7,6.836473,4.5,114.0,-430,1.232,0.811564
3,Tyrese Haliburton,3.6,3.871139,3.5,110.0,-117,1.850,0.540560
4,Quentin Grimes,5.3,5.899639,3.5,-106.0,-526,1.190,0.840296


In [69]:
nba.df_players[['PLAYER_NAME', 'AST', 'expAst', 'ast_line', 'ast_oOdds', 'ASToOdds','ASToOdds_deci','ASToProb']]

,PLAYER_NAME,AST,expAst,ast_line,ast_oOdds,ASToOdds,ASToOdds_deci,ASToProb
0,Coby White,4.6,5.122792,4.5,112.0,-138,1.723,0.580408
1,Josh Hart,5.7,4.989067,5.5,-154.0,160,2.601,0.384492
2,Bennedict Mathurin,1.8,2.067874,1.5,-115.0,-158,1.630,0.613396
3,Tyrese Haliburton,8.5,8.853996,8.5,100.0,-110,1.901,0.525908
4,Quentin Grimes,3.7,4.020083,2.5,105.0,-325,1.307,0.765004


In [70]:
nba.df_players[['PLAYER_NAME', 'PTS', 'expPts', 'pts_line', 'pts_oOdds', 'PTSoOdds','PTSoOdds_deci','PTSoProb']]

,PLAYER_NAME,PTS,expPts,pts_line,pts_oOdds,PTSoOdds,PTSoOdds_deci,PTSoProb
0,Coby White,18.7,18.686267,20.5,-110.0,170,2.707,0.369348
1,Josh Hart,14.4,15.686097,15.5,-110.0,104,2.050,0.487848
2,Bennedict Mathurin,16.3,17.556285,14.5,-105.0,-207,1.483,0.674284
3,Tyrese Haliburton,17.8,21.237693,18.5,-115.0,-171,1.585,0.631112
4,Quentin Grimes,12.3,10.841883,8.5,-130.0,-189,1.527,0.654736


In [16]:
nba.df_players[['PLAYER_NAME', 'PTS', 'expPts', 'pts_line', 'pts_oOdds', 'PTSoOdds','PTSoOdds_deci','PTSoProb']]

,PLAYER_NAME,PTS,expPts,pts_line,pts_oOdds,PTSoOdds,PTSoOdds_deci,PTSoProb
0,Coby White,18.7,18.686267,20.5,-110.0,144.678248,2.447,0.4087
1,Josh Hart,14.4,15.686097,15.5,-110.0,155.558395,2.556,0.3913
2,Bennedict Mathurin,16.3,17.556285,14.5,-105.0,-151.193168,1.661,0.6019
3,Tyrese Haliburton,17.8,21.237693,18.5,-115.0,136.798484,2.368,0.4223
4,Quentin Grimes,12.3,10.841883,8.5,-130.0,-326.803244,1.306,0.7657


In [121]:
nba.df_players.to_excel('test.xlsx')

In [49]:
nba.df_players['expAst'].values

array([5.12279194, 4.98906653, 2.06787421, 8.85399571, 4.02008345])

In [46]:
np.random.poisson(nba.df_players['expAst'].values[:, None], (len(nba.df_players), 10000))

array([[ 1,  7,  8, ...,  6,  7,  4],
       [ 9,  4,  2, ...,  6,  6,  3],
       [ 3,  1,  2, ...,  1,  4,  3],
       [14,  8,  6, ..., 12,  5,  9],
       [ 5,  5,  5, ...,  6,  7,  5]])

In [50]:
(len(nba.df_players), 10000)

(5, 10000)

In [105]:
for i in nba.df_players.columns:
    if 'reb' in i.lower():
        print(i)

OREB
DREB
REB
OREB_PCT
DREB_PCT
OREBadj
DREBadj
reb_line
reb_oOdds
reb_uOdds
OREB_HOMEadj
DREB_HOMEadj
REB_HOMEadj
OREB_ROADadj
DREB_ROADadj
REB_ROADadj
expReb
REBoProb
REBoOdds
REBoOdds_deci


In [63]:
import requests
url_base_nba_player_stat = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID={tid}&TwoWay=0&VsConference=&VsDivision=&Weight='
minute_cutoff = 15.0

team_id = 1610612737
player_info_url = url_base_nba_player_stat.format(
    per = 'PerGame',
    sid = '2024-25', 
    tid = team_id
) 
response = requests.get(url=player_info_url, headers=nba.HEADERS).json()
columns_player_stats = response['resultSets'][0]['headers']

for j in response['resultSets'][0]['rowSet']:
    print(j)

[203992, 'Bogdan Bogdanović', 'Bogdan', 1610612746, 'LAC', 32.0, 24, 12, 12, 0.5, 24.9, 3.5, 9.3, 0.371, 1.8, 6.1, 0.301, 1.3, 1.4, 0.882, 0.5, 2.3, 2.8, 2.0, 1.5, 0.8, 0.3, 0.2, 2.4, 1.6, 10.0, -1.8, 18.1, 0, 0, 18.8, 11, 10, 13, 6, 7, 11, 6, 20, 6, 3, 16, 10, 13, 3, 11, 12, 12, 7, 5, 8, 13, 14, 6, 8, 10, 14, 12, 10, 2, 10]
[1627747, 'Caris LeVert', 'Caris', 1610612737, 'ATL', 30.0, 2, 2, 0, 1.0, 23.7, 4.0, 8.5, 0.471, 2.0, 5.5, 0.364, 3.5, 4.0, 0.875, 0.0, 5.5, 5.5, 1.0, 0.5, 0.0, 0.0, 0.5, 3.5, 2.0, 13.5, -0.5, 21.1, 0, 0, 22.0, 17, 16, 18, 1, 9, 8, 7, 8, 5, 4, 9, 3, 3, 4, 20, 3, 4, 17, 16, 18, 17, 9, 1, 5, 5, 7, 8, 10, 2, 8]
[203991, 'Clint Capela', 'Clint', 1610612737, 'ATL', 30.0, 45, 22, 23, 0.489, 22.4, 4.2, 7.3, 0.573, 0.0, 0.0, 0.0, 1.0, 1.9, 0.541, 3.3, 5.7, 9.0, 1.2, 1.0, 0.6, 1.0, 0.7, 2.0, 1.8, 9.4, -0.8, 25.8, 11, 0, 22.8, 5, 4, 5, 10, 10, 7, 10, 3, 20, 20, 20, 12, 8, 18, 1, 2, 2, 15, 10, 14, 1, 6, 7, 7, 11, 9, 6, 4, 2, 7]
[1631342, 'Daeqwon Plowden', 'Daeqwon', 16106127

In [ ]:
c = nba.df_players.copy()
c['TEAM_ABBREVIATION'].value_counts()[:1].index.tolist()

['BOS']

In [ ]:
c['TEAM_ABBREVIATION'][9:29]

7     ATL
8     ATL
9     ATL
10    ATL
11    ATL
12    ATL
13    ATL
14    BOS
15    BOS
16    BOS
17    BOS
18    BOS
19    BOS
20    BOS
21    CLE
22    CLE
23    CLE
24    CLE
25    CLE
26    CLE
27    CLE
28    NOP
Name: TEAM_ABBREVIATION, dtype: object